In [27]:
from enum import IntEnum, StrEnum
from pathlib import Path

import numpy as np

In [ ]:
input_data = Path("example  .txt").read_text()
lines = input_data.splitlines()


class SpaceString(StrEnum):
    EMPTY = "."
    SPLITTER = "^"
    BEAM = "|"
    START = "S"


class Space(IntEnum):
    EMPTY = 0
    SPLITTER = 1
    BEAM = 2
    START = 3


SPACE_MAP = {
    SpaceString.EMPTY: Space.EMPTY,
    SpaceString.SPLITTER: Space.SPLITTER,
    SpaceString.BEAM: Space.BEAM,
}


def pretty_print(array: np.ndarray) -> None:
    mapping = {
        Space.EMPTY: SpaceString.EMPTY,
        Space.SPLITTER: SpaceString.SPLITTER,
        Space.BEAM: SpaceString.BEAM,
    }
    for i, row in enumerate(array[:, 1:-1]):
        print(
            "".join(
                mapping[val] if (i, j + 1) != start else SpaceString.START
                for j, val in enumerate(row)
            ),
        )


splitters_str = np.array([list(line) for line in lines], dtype=str)
splitters_pad = np.pad(
    splitters_str,
    pad_width=1,
    mode="constant",
    constant_values=SpaceString.EMPTY,
)
splitters_pad = splitters_pad[1:-1]
start = tuple(np.argwhere(splitters_pad == SpaceString.START)[0])
splitters = np.where(
    splitters_pad == SpaceString.EMPTY,
    Space.EMPTY,
    Space.SPLITTER,
)
splitters[start] = Space.BEAM
pretty_print(splitters)

.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............


In [29]:
rows, cols = splitters.shape
total_splits = 0

for index in range(1, rows):
    is_beam = splitters[index - 1] == Space.BEAM
    is_not_split = is_beam & (splitters[index] == Space.EMPTY)
    is_split = is_beam & (splitters[index] == Space.SPLITTER)
    total_splits += is_split.sum()

    split_left = np.roll(is_split, shift=1)
    split_right = np.roll(is_split, shift=-1)
    splitters[index] = np.where(
        is_not_split | split_left | split_right,
        Space.BEAM,
        splitters[index],
    )

splitters[start] = Space.BEAM

print("Total Splits:", total_splits)
pretty_print(splitters)

Total Splits: 21
.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|
